In [14]:
import re
import numpy as np
import heapq
from functools import reduce,cache
from operator import mul
from bisect import insort
from collections import deque
with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
######
#S...#
#..#E#
######
"""

### Part 1
I'm properly sick of 2d grid

In [15]:
# ref:https://medium.com/@nicholas.w.swift/easy-a-star-pathfinding-7e6689c7f7b2
#                          ^ wow! like santa!
class Node():

    def __init__(self, parent=None, position=None, direction=None,cheated=False):
        self.parent = parent
        self.position = position
        self.dir = direction
        self.cheated = False
        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position and self.dir == other.dir and self.cheated == other.cheated
    
    def __lt__(self, other):
        return self.f < other.f
    
    def __str__(self):
        return str((int(self.position.real),int(self.position.imag),self.cheated))
    
    def __hash__(self):
        return hash((self.position,self.dir,self.cheated))

class RaceRace():
    def __init__(self,txt):
        self.grid = txt.splitlines()
        # self.dirs =  {'>':1j,'v':1,'^':-1,'<':-1j}
        self.dirs=[1j,1,-1,-1j]
        self.obs= set()
        self.cost = 0
        # self.end = 0
        # self.start = Node(None,position)
        print(self.grid)
        
        for x in range(len(self.grid)):
            for y in range(len(self.grid[0])):
                # print(x,'',y)
                c=self.grid[x][y]
                if c == '#':
                    self.obs.add(x+y*1j)
                if c == 'S':
                    self.start=Node(None,x+y*1j,1j,cheated=False) # facing east
                if c == 'E':
                    self.end=Node(None,x+y*1j)
        print(f"Start node: {self.start.position}, {self.start.dir}")
        print(f"End node: {self.end.position}")

    def char_at_pos(self,pos):
        return self.grid[int(pos.real)][int(pos.imag)]
    
    def a_star_search(self):
        open,closed = [],{}
        insort(open,self.start)
        total_paths=[]

        # counter = 2000

        while(open):
            # insort(open, item)

            # get current (min(f))
            current = open.pop(0)

            # if found return path
            if current.position == self.end.position:
                print("found")
                path = []
                c = current
                self.cost = c.g
                while c is not None:
                    path.append(c.position)
                    c = c.parent
                total_paths.append(path[::-1]) # Return reversed path
                continue
            
            # add to closed
            if current.position not in closed:
                closed[current.position] = []
            closed[current.position].append(current)


            # define and loop through children
            children, evil_children = [],[]
            for dir in self.dirs:
                new_pos = current.position + dir
                # if new position within bounds
                if 0<=new_pos.real<len(self.grid) and 0<=new_pos.imag<len(self.grid[0]):
                    if new_pos not in self.obs:
                        children.append(Node(current,new_pos,dir,current.cheated))
                if not current.cheated:
                    new_new_pos = new_pos+dir
                    if new_new_pos not in self.obs and new_pos in self.obs and  0<=new_new_pos.real<len(self.grid) and 0<=new_new_pos.imag<len(self.grid[0]):
                        print("AAAAAAAAAAAAAAAAAAAAAAAAAAA")
                        evil_children.append(Node(current,new_new_pos,dir,True)) # TODO: FIX LOGIC

                # if current.cheated == False and new_pos in self.obs and 0<=new_pos.real<len(self.grid) and 0<=new_pos.imag<len(self.grid[0]):
                    
                # if new_pos not in self.obs and 0<=new_pos.real<len(self.grid) and 0<=new_pos.imag<len(self.grid[0]):
                #     children.append(Node(current,new_pos,dir))
           
            for child in children:
                # print(child)
                if child in closed: continue
                
                child.g = current.g + 1
                # # here's the important part: if rotate, +1000
                # if child.dir == current.dir*1j or child.dir == current.dir*-1j: 
                #     # print("turned=", current,child)
                #     child.g += 1000
                child.h = abs(self.end.position.real-child.position.real)+abs(self.end.position.imag-child.position.imag)
                child.f = child.g+child.h
                child.cheated = current.cheated
                
                # if child node doesnt perform better than any node in open
                # just skip to next node
                # for open_node in open:
                #     if child.position == open_node.position and child.g > open_node.g:
                #         continue

                if child.position in closed:
                    if any( closed_node.g <= child.g for closed_node in closed[child.position]):
                        continue
                # in other words, if it is
                insort(open,child)

            for child in evil_children:
                if child in closed: continue
                
                child.g = current.g + 2 # to account for the cheat
                child.h = abs(self.end.position.real-child.position.real)+abs(self.end.position.imag-child.position.imag)
                child.f = child.g+child.h
                child.cheated = True

                if child.position in closed:
                    if any( closed_node.g <= child.g for closed_node in closed[child.position]):
                        continue
                # in other words, if it is
                insort(open,child)

            print(f"Current: {current.position}, g={current.g}, h={current.h}, f={current.f}, cheated={current.cheated}")
            print("Children:")
            for child in children:
                print(f"  {child.position}, g={child.g}, h={child.h}, f={child.f}, cheated={current.cheated}")
            print("Evil Children:")
            for child in evil_children:
                print(f"  {child.position}, g={child.g}, h={child.h}, f={child.f}")
            print("Open list:", [str(node) for node in open])
            # counter-=1
        # Return all found paths
        if total_paths:
            print(f"Found {len(total_paths)} paths to the goal.")
            return total_paths
        else:
            print("No paths found.")
            return None

res = RaceRace(test_input)
paths = res.a_star_search()

# for path in paths:
#     print(path)
#     print(len(path))

# print("part1:", res.cost)


['######', '#S...#', '#..#E#', '######']
Start node: (1+1j), 1j
End node: (2+4j)
Current: (1+1j), g=0, h=0, f=0, cheated=False
Children:
  (1+2j), g=1, h=3.0, f=4.0, cheated=False
  (2+1j), g=1, h=3.0, f=4.0, cheated=False
Evil Children:
Open list: ['(1, 2, False)', '(2, 1, False)']
Current: (1+2j), g=1, h=3.0, f=4.0, cheated=False
Children:
  (1+3j), g=2, h=2.0, f=4.0, cheated=False
  (2+2j), g=2, h=2.0, f=4.0, cheated=False
  (1+1j), g=2, h=4.0, f=6.0, cheated=False
Evil Children:
Open list: ['(2, 1, False)', '(1, 3, False)', '(2, 2, False)']
Current: (2+1j), g=1, h=3.0, f=4.0, cheated=False
Children:
  (2+2j), g=2, h=2.0, f=4.0, cheated=False
  (1+1j), g=2, h=4.0, f=6.0, cheated=False
Evil Children:
Open list: ['(1, 3, False)', '(2, 2, False)', '(2, 2, False)']
Current: (1+3j), g=2, h=2.0, f=4.0, cheated=False
Children:
  (1+4j), g=3, h=1.0, f=4.0, cheated=False
  (1+2j), g=3, h=3.0, f=6.0, cheated=False
Evil Children:
Open list: ['(2, 2, False)', '(2, 2, False)', '(1, 4, False)']
A

In [4]:
class RaceRace:
    def __init__ (self,txt):
        # for i,line in enumerate(txt.splitlines()):
        #     for j ,c in enumerate(line):
        #         if c == '#'
        self.grid = txt.splitlines()                
        self.obs= set()
        self.positions = set()

        for x in range(len(self.grid)):
            for y in range(len(self.grid[x])):
                # print(i,'',j)
                c=self.grid[x][y]
                self.positions = x+y*1j
                if c == '#':
                    self.obs.add(x+y*1j)
                elif c == 'S':
                    self.start = x+y*1j
                elif c == 'E':
                    self.end = x+y*1j



        # self.obs = [tuple(map(int,line.split(","))) for line in txt.splitlines()]
        # self.dirs = [(0,1),(1,0),(-1,0),(0,-1)]
        # self.start = (0,0)
        # self.end = (side-1,side-1)
        # print(f"start={self.start}, end={self.end}")
        # self.positions = set()
        # self.side = side
        # for i in range(side):
        #     for j in range(side):
        #         self.positions.add((i,j))

    # def find_path(self, obstacles):
    #     open = deque([(self.start,0)])
    #     closed = {self.start}
    #     # open.append((self.start,0))
    #     while open:
    #         current_node = open.popleft() # open is a node=(coords,cost)
    #         for neighbor_coords in self.neighbors(current_node[0],obstacles,closed):
    #             if neighbor_coords == self.end:
    #                 return current_node[1]+1
    #             else:
    #                 closed.add(neighbor_coords) # add coords to closed
    #                 open.append((neighbor_coords,current_node[1]+1)) # add {coords,cost}


    def dijkstra(self, obstacles):
        distances = {node: float('inf') for node in self.positions}
        distances[self.start] = 0
        queue = [(0, self.start)]
        while queue:
            current_distance, current_node = heapq.heappop(queue)
            if current_distance > distances[current_node]:
                continue
            for neighbor in self.neighbors(current_node,obstacles):
                # print(neighbor)
                distance = current_distance + 1
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    heapq.heappush(queue, (distance, neighbor))
        return distances # returns a map of all nodes on the map and its shortest distance to start
    
    def neighbors(self,node,obs_set,closed):
        nbs = []
        for dir in self.dirs:
            resX,resY = (dir[0]+node[0]),(dir[1]+node[1])
            new_pos = (resX,resY)
            if 0<=resX<self.side and  0<=resY<self.side and new_pos not in obs_set and new_pos not in closed:
                nbs.append(new_pos) # append new pos (x,y)
        # print(nbs)
        return nbs

        # for i in range(len(self.obs)):
        #     print(f"i={i}, obs={self.obs[i]}")
        #     shortest_path = res.find_path(res.obs[:i+1])
        #     if shortest_path is None: 
        #         print(f"path={shortest_path}, node {self.obs[i]} has blocked the path")
        #         return self.obs[i]

# print(len(input.splitlines()))

# tests
test_res = RaceRace(test_input)
print(test_res.grid)
# assert test_res.find_path(test_res.obs[:12]) == 22
# assert test_res.find_first_block(0,len(test_res.obs)) == (6,1)

# # results
# res = RAMRun(input,71)
# print("part 1:", res.find_path(res.obs[:1024]))
# print("part 2:", res.find_first_block(0,len(res.obs)))

['###############', '#...#...#.....#', '#.#.#.#.#.###.#', '#S#...#.#.#...#', '#######.#.#.###', '#######.#.#...#', '#######.#.###.#', '###..E#...#...#', '###.#######.###', '#...###...#...#', '#.#####.#.###.#', '#.#...#.#.#...#', '#.#.#.#.#.#.###', '#...#...#...###', '###############']
